This notebook is for running NAQS on molecules and reproduction of experimental results/baslines.  It is split into two sections, ***which will set up different environments and so will require the runtime resetting if you want to switch***.

First, however, we'll just pull down and step into the repo itself.

In [2]:
!git clone https://github.com/tomdbar/naqs-for-quantum-chemistry.git

import os
os.chdir("naqs-for-quantum-chemistry")

Cloning into 'naqs-for-quantum-chemistry'...
remote: Enumerating objects: 230, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 230 (delta 21), reused 32 (delta 17), pack-reused 190 (from 1)
Receiving objects: 100% (230/230), 26.86 MiB | 22.96 MiB/s, done.
Resolving deltas: 100% (55/55), done.


# 1. Solve molecules using NAQS

To run this section, we first install OpenFermion and build the necessary cython scripts.

In [3]:
!chmod -R u+x experiments/bash/naqs

try:
    import openfermion as of
except:
    !python -m pip install openfermion==0.11.0
    !python -m pip install openfermionpsi4

!python src_cpp/setup.py build_ext --inplace --force

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.4/631.4 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for openfermion: filename=openfermion-0.11.0-py3-none-any.whl size=741932 sha256=cb3f7234c3909a2bea043d6b0cc5443e2b4bd14978b692c97ca301fa6cd9f767
  Stored in directory: /root/.cache/pip/wheels/da/8a/ed/eab45c8ff1021cde17066d8315e49d9341c8639a88cc8e2b25
  Created wheel for pubchempy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13818 sha256=ec77b221563a2d4ddc8bced85520861759135ee4e9fb0181ee2833a5d2ae4103
  Stored in directory: /root/.cache/pip/wheels/8b/e3/6c/3385b2db08b0985a87f5b117f98d0cb61a3ae3ca3bcbbd8307
Successfully built openfermion pubchempy
Compiling src_cpp/sparse_math.pyx because it changed.
Compiling src_cpp/hilbert_math.pyx because it changed.
Compiling src_cpp/hamiltonian_math.pyx because it changed.
[1/3] Cythonizing src_cpp/hamiltonian_math.pyx
/usr/local/lib/python3.11/dist-packages/Cython/Co

Behind the scence, the script to run specific experiments is ``experiments.run``, however there are many ways to configure this, some of which are not discussed in the paper.  For completeness, the optional arguments can be viewed by running with a ``-h`` or ``--help`` flag.

In [ ]:
!python -u -m experiments.run -h

An example of a configured script, would be this.

In [ ]:
!python -u -m experiments.run -o "data/LiH" -m "molecules/LiH" -single_phase -n1 -n_layer 1 -n_hid 16 -n_layer_phase 1 -n_hid_phase 32 -lr 0.001 -s 1 -n_train 3000 -output_freq 25 -save_freq -1

To ***reproduce the results reported in the paper***, batch training scripts which will perform 5 optimisations with different seeds using all the correct hyperparemeters are provided.  Each script takes the GPU number to use and the molecule name, corresponding to the a sub-folder in ``molecules``.

In [ ]:
!experiments/bash/naqs/batch_train.sh 0 LiH
# !experiments/bash/naqs/batch_train_no_amp_sym.sh 0 LiH
# !experiments/bash/naqs/batch_train_no_mask.sh 0 LiH
# !experiments/bash/naqs/batch_train_full_mask.sh 0 LiH

# 2. Solve molecules using Psi4 (baseline calculations)

First, install Psi4 into a miniconda environment.

In [ ]:
import sys

# Conda env creation and Psi 4 installation.
!wget https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!bash Miniconda3-4.5.4-Linux-x86_64.sh -bfp /usr/local
sys.path.append('/usr/local/lib/python3.6/site-packages')
!conda install -c psi4 psi4 -y

Then run pre-prepared scripts.

In [ ]:
!psi4 experiments/bash/psi4/LiH.dat